# Let's go!
## Imports and Set Up
___

In [ ]:
import holidays
import warnings
from pathlib import Path
from copy import deepcopy  # probably unnecessary

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from joblib import Parallel, delayed

from sklearn.base import (
    BaseEstimator, TransformerMixin, RegressorMixin
)
from sklearn.compose import (
    make_column_transformer, make_column_selector,
)

from sklearn.compose import (
    make_column_transformer,
    TransformedTargetRegressor
)
from sklearn.linear_model import (
    ElasticNet, LinearRegression
)
from sklearn.metrics import (
    mean_absolute_percentage_error,
)
from sklearn.model_selection import (
    cross_val_predict, cross_val_score,
    LeavePGroupsOut, TimeSeriesSplit,
    GridSearchCV
)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (
    MinMaxScaler, OneHotEncoder,
    OrdinalEncoder,
    FunctionTransformer,
    PolynomialFeatures
)
from sklearn.utils.validation import (
    check_X_y, check_array, check_is_fitted
)

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning)

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

sns.set_style("ticks")

INPUT_PATH = Path.cwd().parents[1] / 'kaggle/input/playground-series-s5e1'
TRAIN_PATH = INPUT_PATH / "train.csv"
TEST_PATH = INPUT_PATH / "test.csv"
SUB_PATH = INPUT_PATH / "sample_submission.csv"

## Markdown
___
**Observations** 
* Target has `int` dtype
* Target contains `nan`s
* Target distribution is positively skewed
* Time-wise, *looks* like all test data occur after train data
* No missing values, each entry corresponds to "num_sold" of a given product at the given shop in the given country (5 x 3 x 6 = 90 combinations)
* Looks like evaluation metric does not account for missing values

**Assumptions**
* `nan` in target is equivalent to `0` (i.e., absence is due to lack of sales)
* no hierarchical data

**To Do**
* [X] **EDA**
    * [X] Confirm if test data contains same categories as train data
* [ ] **FE**
    * [ ] Time feature preprocessing
    * [ ] Encode Christmas and other festive seasons in Western countries
    * [ ] Country preprocessing: hemisphere, nordic or not?
    * [ ] Bring basic country data
    * [ ] Decide how to model iso week # (OneHot or Ordinal)
    * [ ] Figure out how to apply rolling/lagging features
* [ ] **Modelling**
    * [ ] Tran without missing entries?
    * [ ] Consider preprocessing target with `TransformedTargetRegressor`
    * [ ] Use `TimeSeriesSplit` or `LeavePGroupsOut` for cross val
    * [ ] Use a baseline model to compare
    * [ ] Try an ensamble of linear models trained on different levels of grouping
    * [ ] Use fallback estimator for `nan`s target
    * [ ] Try an outlier-resistant linear model
    * [ ] Try bayesian models?
    * [X] Consider rounding predictions
    * [ ] 90 linear models using one-hot encoded months, days(weekends?) and normalized years?

In [ ]:
X_data = pd.read_csv(TRAIN_PATH, index_col="date", parse_dates=True)
X_test = pd.read_csv(TEST_PATH, index_col="date", parse_dates=True)
y_test = pd.read_csv(SUB_PATH)

In [ ]:
X_train = X_data.drop(columns=["id", "num_sold"]).copy()
X_test.drop(columns="id", inplace=True)

y_train = X_data["num_sold"].copy()
y_train.fillna(1, inplace=True) ## fill with ones to allow log

## EDA
___
### dtype, nunique, notnulls

In [ ]:
info_df = (
    pd.DataFrame(
        [
            X_train.dtypes,
            X_train.nunique(),
            X_train.notnull().sum(axis=0)
        ],
        index=["dtype", "nunique", "not_null"]
    )
    .T
    .sort_values("nunique", ascending=False)
)
info_df

### Categories

In [ ]:
cat_cols = ["country", "store", "product"]
(
    X_data
    .groupby(cat_cols)["num_sold"]
    .count()
    .to_frame()
    .T
)

### Target

In [ ]:
y_train.describe()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16,3), tight_layout=True)
sns.histplot(
    y_train,
    binrange=(0,6000),
    binwidth=100,
    ax=axes[0]
);
sns.histplot(
    y_train[y_train <= 100],
    discrete=True,
    # binwidth=,
    ax=axes[1]
);
sns.histplot(
    y_train[y_train > 100],
    binrange=(100,5940),
    binwidth=10,
    ax=axes[2]
);
sns.histplot(
    np.log(y_train[y_train > 0]),
    # binrange=(100,5940),
    # binwidth=10,
    ax=axes[3]
);

In [ ]:
def yearly_line_plot(data, color=None):
    resample = data.resample("Y")
    palette = sns.color_palette(
         "Blues", n_colors=resample.ngroups
    )
    shift = 0
    for i, (y, df) in enumerate(resample):
        sns.lineplot(
            df["num_sold"].shift(shift, freq="D"),
            color=palette[i],
            label=y.year
        )
        shift -= df.index.nunique()


In [ ]:
    for country in X_data["country"].unique():
        q_ = (
            f"(country =='{country}')"
            # "and (store == 'Stickers for Less')"
            # "and (product == 'Kaggle')"
        )
        
        g = sns.FacetGrid(
            X_data.query(q_),
            row="product",
            col="store",
            aspect=4,
            height=2
        );
        g.map_dataframe(yearly_line_plot)
        g.add_legend();
        g.figure.suptitle(country, y=1.025)

### Missing Target

In [ ]:
X_data[X_data["num_sold"].isna()].groupby(cat_cols).size()

In [ ]:
def yearly_heatmap(data, **heatmap_kws):
    pre_heat_df = pd.DataFrame(
        np.array([
            data.index.dayofyear,
            data.index.year,
            data["num_sold"]
        ]).T,
        columns=["dayofyear", "year", "num_sold"]
    )
    heat_df = (
        pre_heat_df
        .pivot_table(
            columns="dayofyear", index="year", aggfunc=np.sum
        )
        .replace(0.0, np.nan)
        .droplevel(0, axis=1)
    )
    heat_df.index = heat_df.index.astype(int)
    heat_df.columns = heat_df.columns.astype(int)

    sns.heatmap(
        heat_df,
        **heatmap_kws
    )

In [ ]:
# if RUN_SLOW_CELLS:
for country in ["Canada", "Kenya"]:
    q_ = (
        f"(country =='{country}')"
        "and (store != 'Discount Stickers')"
        "and (product == 'Holographic Goose')"
    )
    
    g = sns.FacetGrid(
        X_data.query(q_),
        # col="country",
        row="store",
        aspect=8,
        height=2
    );
    g.map_dataframe(
        yearly_heatmap,
        vmin=X_data.query(q_)["num_sold"].min(),
        vmax=X_data.query(q_)["num_sold"].max(),
        cbar=False,
        lw=0.005,
        linecolor="k"
    )
    g.add_legend();
    g.figure.suptitle(country, y=1.025)
    g.figure.colorbar(
        g.figure.axes[0].collections[0],
        ax=g.figure.axes,
        orientation='vertical',
        aspect=100,
        fraction=0.025,
        pad=0.01
    );


In [ ]:
mean_absolute_percentage_error(y_train, y_train)

## FE
___

In [ ]:
def transform_date(df):
    return np.array(
        [
            # df.index.year,  # ordinal
            df.index.quarter,
            df.index.month,
            # df.index.isocalendar().week,
            df.index.dayofweek,
            df.index.dayofweek > 4,  # weekend
        ]
    ).T

names_out = [
    "quarter",
    "month",
    # "iso_week",
    "dayofweek",
    "weekend"
]

date_tr = FunctionTransformer(
    transform_date,
    feature_names_out = lambda self, names_in: names_out 
)
date_tr.set_output(transform="pandas")

In [ ]:
date_ohe =  OneHotEncoder(drop="first", sparse_output=False)
date_pl = make_pipeline(date_tr, date_ohe)
date_pl.fit_transform(X_train).shape

In [ ]:
country_holidays = {
    'Canada':    holidays.Canada(),
    'Finland':   holidays.Finland(),
    'Italy':     holidays.Italy(),
    'Kenya':     holidays.Kenya(),
    'Norway':    holidays.Norway(),
    'Singapore': holidays.Singapore(),
}

def transform_holiday(df):
    is_holiday = []
    for idx, row in df.iterrows():
        date = idx  # date is in the index
        ctry = row['country']  # must exist in the DF as a column
        if ctry in country_holidays:
            is_holiday.append(date in country_holidays[ctry])
        else:
            is_holiday.append(False)
    return np.array(is_holiday).reshape(-1, 1)

holiday_tr = FunctionTransformer(
    transform_holiday,
    feature_names_out=lambda self, names_in: ["is_holiday"]
)
holiday_tr.set_output(transform="pandas")

holiday_tr.fit_transform(X_train).value_counts()

In [ ]:
def transform_year(df):
    return np.array(
        [
            df.index.year,
            # df.index.isocalendar().week,
        ]
    ).T

year_tr = FunctionTransformer(
    transform_year,
    feature_names_out = lambda self, names_in: ["year"],
)
year_tr.set_output(transform="pandas")

In [ ]:
year_pl = make_pipeline(
    year_tr,
    PolynomialFeatures(
        degree=3, include_bias=False,
    ), MinMaxScaler()
)
year_pl.fit_transform(X_train).shape

In [ ]:
pre_proc = make_column_transformer(
    (OrdinalEncoder(dtype=int), cat_cols),  # for grouping by
    (year_pl, cat_cols),
    (date_pl, cat_cols),
    (holiday_tr, cat_cols)
)
pre_proc.set_output(transform="pandas")
X_train_pp = pre_proc.fit_transform(X_train)
X_test_pp = pre_proc.transform(X_test)

In [ ]:
X_test_pp.head().T

## Modelling
___

In [ ]:
class GroupRegression(BaseEstimator, RegressorMixin):
    """
    A scikit-learn style estimator that fits separate regression models 
    for each unique combination of categorical columns (groupby_cols).
    
    Parameters
    ----------
    groupby_cols : list
        Column names in X to use for grouping. A separate model will be
        fit for each unique combination of these columns.

    base_estimator : estimator, default=None
        If None, uses ElasticNet(**base_estimator_kws) as the default model.
        Otherwise, use your own regressor.

    n_jobs : int, default=1
        Number of CPU cores for parallel fitting.

    **base_estimator_kws : dict
        Additional keyword args passed to the default ElasticNet if base_estimator is None.
    """
    def __init__(
        self,
        groupby_cols,
        base_estimator=None,
        n_jobs=-1,
        **base_estimator_kws
    ):
        self.n_jobs = n_jobs
        self.groupby_cols = groupby_cols
        if base_estimator is None:
            self.base_estimator = ElasticNet(**base_estimator_kws)
        else:
            self.base_estimator = base_estimator
            self.base_estimator.set_params(**base_estimator_kws)
        self.base_estimator_kws = base_estimator_kws

    def fit(self, X, y):
        """Fit separate estimators for each group."""
        check_X_y(X, y, dtype=None)  # Allow non-numerical grouping columns
        X = pd.DataFrame(X).reset_index(drop=True)
        y = pd.Series(y.values, index=X.index)

        for col in self.groupby_cols:
            if col not in X.columns:
                raise KeyError(f"X does not contain the grouping column: {col}")

        self.n_features_in_ = X.shape[1] - len(self.groupby_cols)
        self.estimators_ = {}

        def fit_one_group(group_key, df):
            estimator = deepcopy(self.base_estimator)
            X_local = df.drop(columns=self.groupby_cols)
            y_local = y.loc[df.index]
            estimator.fit(X_local, y_local)
            return (group_key, estimator)

        results = Parallel(n_jobs=self.n_jobs)(
            delayed(fit_one_group)(g, df) for g, df in X.groupby(self.groupby_cols)
        )

        for group_key, estimator in results:
            self.estimators_[group_key] = estimator

        return self

    def predict(self, X):
        """Predict using the group-specific model if available; else default to zeros."""
        check_is_fitted(self, 'estimators_')
        X = pd.DataFrame(X).reset_index(drop=True)

        for col in self.groupby_cols:
            if col not in X.columns:
                raise KeyError(f"X does not contain the grouping column: {col}")

        y_pred = np.zeros(X.shape[0], dtype=float)

        def predict_one_group(group_key, df):
            if group_key in self.estimators_:
                est = self.estimators_[group_key]
                return est.predict(df.drop(columns=self.groupby_cols))
            else:
             # return defaults for Hologoose in Kenya and Canada
                default = np.ones(len(df), dtype=float)
                if "Canada" in group_key:
                    default *= 200
                if "Kenya" in group_key:
                    default *= 5
                return default

        for group_key, df in X.groupby(self.groupby_cols):
            idx = df.index
            y_pred[idx] = predict_one_group(group_key, df)

        return y_pred

    def get_params(self, deep=True):
        """Return parameters, including nested base_estimator params."""
        params = super().get_params(deep=False)
        if deep and hasattr(self.base_estimator, 'get_params'):
            for k, v in self.base_estimator.get_params(deep=True).items():
                params[f'base_estimator__{k}'] = v
        return params

    def set_params(self, **params):
        """Set parameters, parsing out base_estimator__ params."""
        base_estimator_params = {}
        for key, val in list(params.items()):
            if key.startswith('base_estimator__'):
                base_estimator_params[key[len('base_estimator__'):]] = val
                del params[key]
        super().set_params(**params)
        if base_estimator_params and hasattr(self.base_estimator, 'set_params'):
            self.base_estimator.set_params(**base_estimator_params)
        return self

In [ ]:
class ResidualEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, model, cols):
        self.model = deepcopy(model)
        self.cols = cols.copy()

    def fit(self, X, y):
        def q_group(s):
            qcut_ = pd.qcut(s, 11, labels=np.arange(-1,1.2, 0.2))
            qcut_.name = "qcut"
            return qcut_.astype(float)
            
        X_ = X.copy()
        self.model.fit(X_, y)
        X_["residual"] = y - self.model.predict(X)
        X_["doy"] = X_.index.dayofyear
        self.mean_ = (
            X_
            .groupby(self.cols + ["doy"])["residual"]
            .mean()
            .to_frame("resid_mean")
        )
        self.qcut_ = (
            self.mean_
            .groupby(self.cols)["resid_mean"]
            .transform(q_group)
            .to_frame("resid_qbin")
        )
        return self
        
    def transform(self, X):
        X_ = X.copy()
        idx_ = X_.index
        X_["doy"] = X_.index.dayofyear
        X_ = pd.merge(
            X_, self.qcut_,
            on=self.cols + ["doy"],
            how="left"
        )
        return (
            X_
            .drop(columns="doy")
            .fillna(0)
            .set_index(idx_)
        )

In [ ]:
 groupby_cols = [
    "ordinalencoder__country",
    "ordinalencoder__store",
    "ordinalencoder__product"
]

gr = GroupRegression(groupby_cols, n_jobs=-1, alpha=.1)

In [ ]:
re = ResidualEncoder(gr, groupby_cols)

In [ ]:
notnull_mask = X_data["num_sold"].notnull()

In [ ]:
re.fit(X_train_pp[notnull_mask], y_train[notnull_mask])

In [ ]:
X_train_pp = re.transform(X_train_pp)
X_test_pp = re.transform(X_test_pp)

### Validation 1

In [ ]:
# "2010":"2014"
gr.fit(
    X_train_pp.loc[notnull_mask].loc["2010":"2014"],
    y_train.loc[notnull_mask].loc["2010":"2014"]
)

In [ ]:
y_pred = gr.predict(X_train_pp[notnull_mask].loc["2015":])
y_true = y_train[notnull_mask].loc["2015":]
mean_absolute_percentage_error(y_true, y_pred)

### Validation 2

In [ ]:
def custom_time_splits(X, notnull_mask=None, train_on_null=True):
    """
    Generate 5 time-based splits:
      - Train = 3 years
      - Validation = 0.5 year
      - Test = 1.5 years
    Each split is shifted by 0.5 year from the previous split.

    Returns:
      An iterator of (train_idx, val_idx, test_idx) tuples.
    """

    # The earliest date in the dataset
    min_date = X.index[0]

    if notnull_mask is None:
        notnull_mask = pd.Series([True] * X.shape[0])

    # We'll define lengths in months:
    train_months = 36   # 3 years
    val_months   = 6    # 0.5 year
    test_months  = 18   # 1.5 years
    shift_months = 6    # shift each split by 0.5 year

    # We'll produce 4 splits total
    for i in range(5):
        # Compute start/end boundaries for each window
        train_start = min_date + pd.DateOffset(months=shift_months * i)
        train_end   = train_start + pd.DateOffset(months=train_months)

        val_start   = train_end
        val_end     = val_start + pd.DateOffset(months=val_months)

        test_start  = val_end
        test_end    = test_start + pd.DateOffset(months=test_months)

        # Create boolean masks
        train_mask = (X.index >= train_start) & (X.index < train_end)
        if not train_on_null:
            train_mask &= notnull_mask
        val_mask   = (X.index >= val_start)   & (X.index < val_end) & notnull_mask
        test_mask  = (X.index >= test_start)  & (X.index < test_end) & notnull_mask

        # Convert masks to integer indices
        train_idx = np.where(train_mask)[0]
        val_idx   = np.where(val_mask)[0]
        test_idx  = np.where(test_mask)[0]

        yield (train_idx, val_idx, test_idx)


In [ ]:
def evaluate_time_splits(
    X, y, splitter, model_factory,
    metric=mean_absolute_percentage_error
):
    """
    Iterates over time-based splits, trains a model for each split,
    logs train/validation/test scores, and returns a results DataFrame.
    """
    cv_records = []
    
    for i, (train_idx, val_idx, test_idx) in enumerate(splitter, start=1):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val,   y_val   = X.iloc[val_idx],   y.iloc[val_idx]
        X_test,  y_test  = X.iloc[test_idx],  y.iloc[test_idx]

        model = model_factory()
        model.fit(X_train, y_train)

        train_score = metric(y_train, model.predict(X_train))
        val_score   = metric(y_val,   model.predict(X_val))
        test_score  = metric(y_test,  model.predict(X_test))

        cv_records.append({
            'split': i,
            'train_score': train_score,
            'val_score': val_score,
            'test_score': test_score
        })
    
    results_df = pd.DataFrame(cv_records)
    # for col in ["train_score", "val_score", ""]
    return results_df


In [ ]:
splitter_ = custom_time_splits(
    X_train_pp,
    notnull_mask=notnull_mask,
    train_on_null=True
)
results_df = evaluate_time_splits(
    X_train_pp, y_train, splitter_,
    lambda: deepcopy(gr)
)
results_df

In [ ]:
results_df.mean()

## Diagnostics
___

In [ ]:
fig, ax =plt.subplots(figsize=(16,9))
df = pd.DataFrame(
    [e.coef_ for e in gr.estimators_.values()],
    index=gr.estimators_.keys(),
    columns=X_train_pp.columns[3:]
)
sns.heatmap(df, ax=ax, cmap="Spectral_r", center=0);

In [ ]:
maep_ = abs(y_true - y_pred) / y_true
maep_.hist();

In [ ]:
def yearly_maep_heatmap(data, **heatmap_kws):
    pre_heat_df = pd.DataFrame(
        np.array([
            data.index.dayofyear,
            data.index.year,
            data["maep"]
        ]).T,
        columns=["dayofyear", "year", "maep"]
    )
    heat_df = (
        pre_heat_df
        .pivot_table(
            columns="dayofyear", index="year", aggfunc=np.mean
        )
        # .replace(0.0, np.nan)
        .droplevel(0, axis=1)
    )
    heat_df.index = heat_df.index.astype(int)
    heat_df.columns = heat_df.columns.astype(int)
    heat_df = heat_df.reindex(range(1,367), axis=1)

    ax = sns.heatmap(
        heat_df,
        **heatmap_kws
    )


In [ ]:
# if RUN_SLOW_CELLS:
for country in ["Kenya"]:
    q_ = (
        f"(country =='{country}')"
        "and (store == 'Stickers for Less')"
        "and (product == 'Holographic Goose')"
    )
    data = (
        X_data[notnull_mask]
        .loc["2015":]
        .assign(maep=maep_.values)
        .query(q_)
    )
    g = sns.FacetGrid(
        data,
        col="store",
        row="product",
        aspect=8,
        height=2
    );
    g.map_dataframe(
        yearly_maep_heatmap,
        vmin=data["maep"].min(),
        vmax=data["maep"].max(),
        cbar=False,
        lw=0.005,
        linecolor="k",
        cmap="jet"
    )
    g.add_legend();
    g.figure.suptitle(country, y=1.025)
    g.figure.colorbar(
        g.figure.axes[0].collections[0],
        ax=g.figure.axes,
        orientation='vertical',
        aspect=25,
        fraction=0.025,
        pad=0.01
    );


## Submission
___

In [ ]:
gr.fit(X_train_pp.loc[notnull_mask], y_train.loc[notnull_mask])
y_test["num_sold"] = gr.predict(X_test_pp)
y_test.to_csv('submission.csv', index=False)